# Interacting with the infrastructure with your own credentials

This code shows how to get a valid access token to interact 
with Check-in integrated services. 

**Beware the API for refreshing tokens is still under development 
and may change in the future**

In this example we use the token to list the VM on one of the providers of the EGI Cloud

In [1]:
# Install OpenStack client
%pip install -q keystoneauth1 python-novaclient tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import os

api_url = os.environ.get('JUPYTERHUB_API_URL')
token = os.environ.get('JUPYTERHUB_API_TOKEN')

r = requests.get(api_url + '/refresh',
    headers={
             'Authorization': 'token %s' % token,
            }
    )
checkin_token = r.json()['access_token']
print("Got an access token: %s..." % checkin_token[0:10])

Got an access token: eyJraWQiOi...


In [3]:
# Now use that token into another service
# Shown as example one provider of the EGI Cloud

from keystoneauth1 import session
from keystoneauth1.identity import v3
import novaclient.client

# This is from GOCDB
auth_url="https://sbgcloud.in2p3.fr:5000/v3"
project_id="a5eb30bba2c2497b90645fb199e34b39"

s = session.Session(auth=v3.OidcAccessToken(auth_url=auth_url,
                                            identity_provider="egi.eu",
                                            protocol="openid",
                                            access_token=checkin_token,
                                            project_id=project_id))
nova = novaclient.client.Client(2, session=s)


servers_table = []
for server in nova.servers.list():
    servers_table.append((server.id, server.name, server.status))

from tabulate import tabulate

print(tabulate(servers_table, headers=("Server ID", "Name", "Status")))

Server ID                             Name             Status
------------------------------------  ---------------  --------
70f7ca69-e488-4ba4-b6e4-6775694e18e6  ocims-01         ACTIVE
6cc317e0-a56b-418d-b451-d95c0a570c3a  sectest-ext-net  ACTIVE
46fc3784-b592-4493-88fa-0b1c1459a7eb  sectest          ACTIVE
a09fa34e-4479-455e-a9cb-deaadae5a0c5  Kubernetes       ACTIVE
76863ff3-02c2-45c9-901e-7966031c73ad  Kubernetes       ACTIVE
6fe857b8-d66f-4d74-857c-b5d5bb42d53d  Kubernetes       ACTIVE
4435dea2-5b29-458e-a224-460a5d9dda3e  Kubernetes       ACTIVE
22056396-12cd-4038-8831-ad17ba3a99d1  Kubernetes       ACTIVE
19faffdb-4a1c-4367-8494-0d339b04bbc4  infopro          ACTIVE
5b7d02c3-d0df-4c47-ab4c-337089ff91c7  im_userimage     ACTIVE
